# ACCESS MET File Creation

## Overview

This notebook creates an ACCESS APSIM MET file by merging data from:

1. **Base variables** (tasmax, tasmin, pr, rsds) - extracted from CMIP6 NetCDF files
   - **Caching:** Extracted data is automatically cached to CSV files for faster subsequent runs
2. **Vapor Pressure (VP)** - loaded from CSV file generated by `CMPI6_VP_Calculation.ipynb`
3. **Evaporation (Evap)** - loaded from CSV file generated by `CMPI6_Evap_Calculation.ipynb` (which already incorporates sfcWind)

## Output

ACCESS MET file with all required variables:
- **maxt**: Maximum temperature (°C)
- **mint**: Minimum temperature (°C)
- **rain**: Precipitation (mm)
- **radn**: Radiation (MJ/m²)
- **evap**: Class A pan evaporation (mm/day)
- **vp**: Vapor pressure (hPa)
- **code**: Hardcoded to '222222'

## Configuration

Only change scenario, coordinates, and output directory at the top. All other settings adjust automatically.

## Section 1: Imports and Configuration

In [49]:
import pandas as pd
import numpy as np
import xarray as xr
import glob
import os
import time
import re
import math
import calendar
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

print("Libraries imported successfully")

Libraries imported successfully


In [50]:
# CONFIGURATION - CHANGE VALUES BELOW AS NEEDED
# ============================================================================
# All other settings will automatically adjust based on these values

# Model (usually doesn't need to change)
MODEL = "ACCESS CM2"  # e.g., "ACCESS CM2"

# Scenario - CHANGE THIS
SCENARIO = "SSP585"   # Options: "obs", "SSP245", "SSP585", etc.

# Coordinates - CHANGE THESE
LATITUDE = -31.75   # Target latitude in decimal degrees (-90 to 90)
LONGITUDE = 117.60  # Target longitude in decimal degrees (-180 to 180)

# Date Range - CHANGE THESE
START_YEAR = 1986   # Start year for MET file (e.g., 1985 for obs, 2035 for future scenarios)
END_YEAR = 2014     # End year for MET file (e.g., 2014 for obs, 2064 for future, or None for all available data)

# Output Directory - OPTIONAL: Set to None to auto-generate, or specify a custom path
OUTPUT_DIR_MANUAL = r"C:\Users\ibian\Desktop\ClimAdapt\Anameka\Anameka Climate Files_2901\-31.75_117.60_Climate Files"  # Set to None for auto-generation

# ============================================================================
# AUTOMATIC SETTINGS (derived from above - no need to change)
# ============================================================================

# Base directories
CMIP6_BASE_DIR = r"C:\Users\ibian\Desktop\ClimAdapt\CMIP6"
base_output_dir = r"C:\Users\ibian\Desktop\ClimAdapt\Anameka"
COORD_TOLERANCE = 0.01  # degrees (approximately 1.1 km)

# Auto-generate output directory and filename components based on scenario and coordinates
lat_str = f"{LATITUDE:.2f}".replace('.', '_').replace('-', 'neg')
lon_str = f"{LONGITUDE:.2f}".replace('.', '_').replace('-', 'neg')
model_scenario = f"{MODEL.replace(' ', '_')}_{SCENARIO}"
model_scenario_dir = f"{model_scenario}_{lat_str}_{lon_str}"

# Use manual output directory if specified, otherwise auto-generate
if OUTPUT_DIR_MANUAL is not None and OUTPUT_DIR_MANUAL != "":
    OUTPUT_DIR = OUTPUT_DIR_MANUAL
    print(f"  [INFO] Using manual output directory: {OUTPUT_DIR}")
else:
    OUTPUT_DIR = os.path.join(base_output_dir, model_scenario_dir)
    print(f"  [INFO] Auto-generated output directory: {OUTPUT_DIR}")

# Variables required for ACCESS MET file
# Base variables (from CMIP6)
BASE_VARIABLES = ['tasmax', 'tasmin', 'pr', 'rsds']
# Variables for VP calculation
VP_VARIABLES = ['hurs']  # VP calculated from hurs, tasmax, tasmin
# Variables for Evap calculation
# Note: sfcWind is not extracted here - it's already used in CMPI6_Evap_Calculation.ipynb
EVAP_VARIABLES = ['hurs']  # hurs used for evap calculation (sfcWind already incorporated in evap CSV)

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("="*70)
print("CONFIGURATION")
print("="*70)
print(f"  Model: {MODEL}")
print(f"  Scenario: {SCENARIO}")
print(f"  Coordinates: ({LATITUDE:.6f}, {LONGITUDE:.6f})")
print(f"  Date Range: {START_YEAR} to {END_YEAR if END_YEAR is not None else 'end of data'}")
print(f"  CMIP6 Base Directory: {CMIP6_BASE_DIR}")
print(f"  Output Directory: {OUTPUT_DIR}")
print(f"  Base Variables: {', '.join(BASE_VARIABLES)}")
print(f"  VP Variables: {', '.join(VP_VARIABLES)}")
print(f"  Evap Variables: {', '.join(EVAP_VARIABLES)}")
print("="*70)
print("\nAll paths and filenames will automatically use the above settings.\n")

  [INFO] Using manual output directory: C:\Users\ibian\Desktop\ClimAdapt\Anameka\Anameka Climate Files_2901\-31.75_117.60_Climate Files
CONFIGURATION
  Model: ACCESS CM2
  Scenario: SSP585
  Coordinates: (-31.750000, 117.600000)
  Date Range: 1986 to 2014
  CMIP6 Base Directory: C:\Users\ibian\Desktop\ClimAdapt\CMIP6
  Output Directory: C:\Users\ibian\Desktop\ClimAdapt\Anameka\Anameka Climate Files_2901\-31.75_117.60_Climate Files
  Base Variables: tasmax, tasmin, pr, rsds
  VP Variables: hurs
  Evap Variables: hurs

All paths and filenames will automatically use the above settings.



## Section 2: NetCDF Data Extraction Function

In [51]:
def extract_daily_data_from_netcdf(netcdf_dir, variable, target_lat, target_lon, tolerance=0.01):

    start_time = time.time()
    
    # Find all NetCDF files in the directory
    nc_files = sorted(glob.glob(os.path.join(netcdf_dir, f"*{variable}*.nc")))
    
    # Pattern 2: Files in subdirectories named {variable}_*
    if len(nc_files) == 0:
        var_subdirs = glob.glob(os.path.join(netcdf_dir, f"{variable}_*"))
        for var_subdir in var_subdirs:
            if os.path.isdir(var_subdir):
                found_files = sorted(glob.glob(os.path.join(var_subdir, "*.nc")))
                if found_files:
                    nc_files.extend(found_files)
                    print(f"  Found files in subdirectory: {os.path.basename(var_subdir)}/")
                    break
    
    # For rsds, also check rad_* folders
    if len(nc_files) == 0 and variable == 'rsds':
        rad_subdirs = glob.glob(os.path.join(netcdf_dir, "rad_*"))
        for rad_subdir in rad_subdirs:
            if os.path.isdir(rad_subdir):
                found_files = sorted(glob.glob(os.path.join(rad_subdir, "*rsds*.nc")))
                if found_files:
                    nc_files.extend(found_files)
                    print(f"  Found files in subdirectory: {os.path.basename(rad_subdir)}/")
                    break
    
    # For sfcWind, also check wind_* folders
    if len(nc_files) == 0 and variable == 'sfcWind':
        wind_subdirs = glob.glob(os.path.join(netcdf_dir, "wind_*"))
        for wind_subdir in wind_subdirs:
            if os.path.isdir(wind_subdir):
                found_files = sorted(glob.glob(os.path.join(wind_subdir, "*sfcWind*.nc")))
                if found_files:
                    nc_files.extend(found_files)
                    print(f"  Found files in subdirectory: {os.path.basename(wind_subdir)}/")
                    break
        # If no files found with sfcWind in name, try any .nc files in wind_* directory
        if len(nc_files) == 0:
            for wind_subdir in wind_subdirs:
                if os.path.isdir(wind_subdir):
                    found_files = sorted(glob.glob(os.path.join(wind_subdir, "*.nc")))
                    if found_files:
                        nc_files.extend(found_files)
                        print(f"  Found files in subdirectory: {os.path.basename(wind_subdir)}/")
                        break
    
    if len(nc_files) == 0:
        print(f"  ERROR: No NetCDF files found in {netcdf_dir}")
        return None
    
    print(f"  Found {len(nc_files)} NetCDF files")
    
    # Cache coordinate information from first file
    lat_name = None
    lon_name = None
    time_name = None
    lat_idx = None
    lon_idx = None
    var_name = None
    
    # List to store daily data
    all_data = []
    
    # Process first file to get coordinate structure
    if len(nc_files) > 0:
        try:
            ds_sample = xr.open_dataset(nc_files[0], decode_times=False)
            
            # Get variable name
            for v in ds_sample.data_vars:
                if variable in v.lower() or v.lower() in variable.lower():
                    var_name = v
                    break
            
            if var_name is None:
                possible_names = [variable, variable.upper(), f'{variable}_day']
                if variable == 'rsds':
                    possible_names.extend(['rad', 'RAD', 'rad_day'])
                for name in possible_names:
                    if name in ds_sample.data_vars:
                        var_name = name
                        break
            
            # Get coordinate names
            for coord in ds_sample.coords:
                coord_lower = coord.lower()
                if 'lat' in coord_lower:
                    lat_name = coord
                elif 'lon' in coord_lower:
                    lon_name = coord
                elif 'time' in coord_lower:
                    time_name = coord
            
            if lat_name and lon_name:
                # Find nearest grid point
                lat_idx = np.abs(ds_sample[lat_name].values - target_lat).argmin()
                lon_idx = np.abs(ds_sample[lon_name].values - target_lon).argmin()
                
                actual_lat = float(ds_sample[lat_name].values[lat_idx])
                actual_lon = float(ds_sample[lon_name].values[lon_idx])
                
                # Check if within tolerance
                if abs(actual_lat - target_lat) > tolerance or abs(actual_lon - target_lon) > tolerance:
                    print(f"  Warning: Nearest point ({actual_lat:.4f}, {actual_lon:.4f}) is outside tolerance")
                else:
                    print(f"  Using grid point: ({actual_lat:.4f}, {actual_lon:.4f})")
            
            ds_sample.close()
            
        except Exception as e:
            print(f"  Warning: Could not read sample file: {e}")
    
    if var_name is None or lat_idx is None or lon_idx is None:
        print(f"  ERROR: Could not determine coordinate structure")
        return None
    
    # Process all files with progress bar
    print(f"  Processing files...")
    for nc_file in tqdm(nc_files, desc=f"  {variable}", unit="file"):
        try:
            ds = xr.open_dataset(nc_file, decode_times=False)
            
            # Extract data using cached indices
            data = ds[var_name].isel({lat_name: lat_idx, lon_name: lon_idx})
            
            # Convert to numpy array
            values = data.values
            if values.ndim > 1:
                values = values.flatten()
            
            # Get time values - extract year from filename
            year = None
            filename = os.path.basename(nc_file)
            year_match = re.search(r'(\d{4})', filename)
            if year_match:
                year = int(year_match.group(1))
                # Create daily dates for the year (handles leap years automatically)
                time_values = pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31', freq='D')
            else:
                time_values = pd.date_range(start='2035-01-01', periods=len(values), freq='D')
            
            # Ensure correct number of dates
            if len(time_values) != len(values):
                if len(time_values) > len(values):
                    time_values = time_values[:len(values)]
            
            # Create DataFrame for this file
            if len(values) > 0:
                df_file = pd.DataFrame({
                    'date': time_values[:len(values)],
                    'value': values
                })
                all_data.append(df_file)
            
            ds.close()
            
        except Exception as e:
            tqdm.write(f"    Error processing {os.path.basename(nc_file)}: {e}")
            continue
    
    if len(all_data) == 0:
        print(f"  ERROR: No data extracted")
        return None
    
    # Combine all data
    print(f"  Combining data from {len(all_data)} files...")
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Sort by date
    combined_df = combined_df.sort_values('date').reset_index(drop=True)
    
        # Remove duplicate dates (keep first occurrence)
    combined_df = combined_df.drop_duplicates(subset='date', keep='first')
    
    # Check for constant values by year (indicates missing/corrupted data in source files)
    combined_df['year'] = pd.to_datetime(combined_df['date']).dt.year
    for year in combined_df['year'].unique():
        year_data = combined_df[combined_df['year'] == year]['value']
        non_null_data = year_data.dropna()
        if len(non_null_data) > 10:  # Only check if we have enough data points
            if non_null_data.nunique() == 1:
                constant_value = non_null_data.iloc[0]
                num_days = len(non_null_data)
                print(f"  [WARNING] Year {year} has constant {variable} value: {constant_value:.2f} for {num_days} days (likely missing/corrupted source data)")
    combined_df = combined_df.drop(columns=['year'])
    
    elapsed_time = time.time() - start_time
    print(f"  ✓ Extracted {len(combined_df):,} daily records in {elapsed_time:.1f} seconds")
    print(f"  Date range: {combined_df['date'].min()} to {combined_df['date'].max()}")
    
    return combined_df

## Section 3: Caching Functions

To optimize performance, extracted NetCDF data is cached to CSV files. On subsequent runs, 
cached data is loaded automatically instead of re-extracting from NetCDF files.

**Cache Location:** Cached files are saved in the output directory with naming convention:
`{model_scenario}_{lat_str}_{lon_str}_{variable}.csv`

**Cache Behavior:**
- If cache exists and is valid → Load from cache (fast)
- If cache doesn't exist or is invalid → Extract from NetCDF files and save to cache


## Section 3: Load VP and Evap Data from Calculation Notebooks

This notebook loads Vapor Pressure (VP) and Evaporation (Evap) data from the CSV files generated by:
- **`CMPI6_VP_Calculation.ipynb`** - Calculates VP using SILO method
- **`CMPI6_Evap_Calculation.ipynb`** - Calculates Evap using FAO-56 Penman-Monteith with SILO calibration

**Important:** Before running this notebook, ensure that:
1. `CMPI6_VP_Calculation.ipynb` has been executed and generated the VP CSV file
2. `CMPI6_Evap_Calculation.ipynb` has been executed and generated the Evap CSV file

The CSV files are expected to be in the same output directory and follow the naming convention:
- VP: `{model_scenario}_{lat_str}_{lon_str}_vp.csv`
- Evap: `{model_scenario}_{lat_str}_{lon_str}_evap.csv`

Where:
- `model_scenario` = `{MODEL.replace(' ', '_')}_{SCENARIO}` (e.g., "ACCESS_CM2_SSP245")
- `lat_str` = latitude formatted as string (e.g., "neg31_75")
- `lon_str` = longitude formatted as string (e.g., "117_60")

### Vapor Pressure (VP) Calculation - SILO Method

**Input Variables:**
- **hurs**: Mean relative humidity (%) - required
- **tasmax**: Daily maximum temperature (°C) - required
- **tasmin**: Daily minimum temperature (°C) - required

**Output:**
- **vp**: Vapor pressure (hPa) matching SILO units

**Calculation Method:**

#### Saturation Vapor Pressure (kPa)

For mean temperature T_mean in °C:

```
e_s(T_mean) = 0.611 × exp(17.27 × T_mean / (T_mean + 237.3))
```

Where: T_mean = (tasmax + tasmin) / 2

#### Actual Vapor Pressure (hPa)

Using mean relative humidity (hurs) and saturation vapor pressure:

```
VP(hPa) = 10 × (hurs/100) × e_s(T_mean)
```

Or directly in hPa:

```
VP(hPa) = (hurs/100) × 0.611 × exp(17.27 × T_mean / (T_mean + 237.3)) × 10
```

This gives a daily VP proxy. SILO VP is "9am-like"; to replicate SILO closely, you would bias-correct this VP proxy to SILO over a historical overlap period.

---

### Evaporation (Evap) Calculation - FAO-56 Penman-Monteith

**Key Principle:**

Class A pan evaporation cannot be directly calculated from CMIP6. Instead, we:
- Compute a physically based evaporation proxy (ET₀) from CMIP6 meteorology
- Calibrate (bias-correct) it to SILO pan evaporation over a historical overlap period

This is the standard and accepted approach in Australian crop modelling.

**Input Variables (CMIP6):**
- **tasmax**: Daily maximum temperature (°C) - required
- **tasmin**: Daily minimum temperature (°C) - required
- **hurs**: Daily mean relative humidity (%) - required
- **rsds**: Surface downwelling shortwave radiation (W/m²) - required
- **sfcWind**: Wind speed at 10m (m/s) - used in `CMPI6_Evap_Calculation.ipynb` (already incorporated in evap CSV)

**Output:**
- **evap**: Class A pan evaporation (mm/day) - SILO-scaled and APSIM-compatible

**Calculation Method:**

#### Step 1: Calculate ET₀ (FAO-56 Penman-Monteith)

ET₀ (mm/day) = physically based evaporation driven by radiation + temperature + humidity + wind

Formula:
```
ET₀ = (0.408 × Δ × (Rn - G) + γ × (900/(T+273)) × u₂ × (es - ea)) / (Δ + γ × (1 + 0.34 × u₂))
```

Where:
- `Δ` = slope of vapor pressure curve (kPa/°C) = 4098 × es / (T + 237.3)²
- `Rn` = net radiation (MJ/m²/day) ≈ 0.77 × Rs (where Rs is solar radiation converted from W/m² to MJ/m²/day)
- `G` = soil heat flux (MJ/m²/day) - assumed 0 for daily calculations
- `γ` = psychrometric constant (kPa/°C) = 0.665e-3 × 101.3 ≈ 0.0675 kPa/°C
- `T` = mean air temperature (°C) = (tasmax + tasmin) / 2
- `u₂` = wind speed at 2m height (m/s) - converted from 10m using logarithmic wind profile
- `es` = saturation vapor pressure (kPa) at mean temperature
- `ea` = actual vapor pressure (kPa) calculated from hurs (mean relative humidity), tasmax, tasmin

**Wind Speed Conversion (10m to 2m):**
```
u₂ = u₁₀ × ln(2/z₀) / ln(10/z₀)
```
Where z₀ = 0.0002 m (roughness length for open water/pan)

**Actual Vapor Pressure (for Evap):**
```
ea = es(T_mean) × hurs / 100
```
Where T_mean = (tasmax + tasmin) / 2

#### Step 2: Monthly Calibration to SILO Pan Evaporation

For each calendar month:
```
pan_evap = a_m × ET₀ + b_m
```

Where `a_m` and `b_m` are fitted using SILO evap_pan over the historical overlap period (1981-2010 or 1991-2020).

#### Step 3: Apply Calibration to Future Projections

Compute ET₀ using the same formulas, then apply the monthly calibration coefficients.

**Justification:**

"Daily pan evaporation was derived from CMIP6 meteorological variables by first calculating FAO-56 reference evapotranspiration and then bias-correcting to SILO Class A pan evaporation over a historical overlap period, ensuring consistency with APSIM soil-water and crop-growth parameterisation."

In [52]:
def get_cached_variable_path(output_dir, model_scenario, lat_str, lon_str, variable):
    """
    Generate the path for a cached variable CSV file.
    Tries multiple naming formats to match existing cache files.
    
    Parameters:
    -----------
    output_dir : str
        Output directory
    model_scenario : str
        Model and scenario string (e.g., "ACCESS_CM2_SSP245")
    lat_str : str
        Latitude formatted as string (e.g., "neg31_45")
    lon_str : str
        Longitude formatted as string (e.g., "117_55")
    variable : str
        Variable name (e.g., "tasmax", "hurs")
    
    Returns:
    --------
    str
        Path to cached CSV file (tries decimal format first, then underscore format)
    """
    # Try decimal format first (matches existing files: -31.45_117.55)
    # Convert from underscore format to decimal format
    if 'neg' in lat_str:
        lat_decimal = lat_str.replace('neg', '-').replace('_', '.')
    elif '_' in lat_str:
        lat_decimal = lat_str.replace('_', '.')
    else:
        lat_decimal = lat_str
    
    if '_' in lon_str:
        lon_decimal = lon_str.replace('_', '.')
    else:
        lon_decimal = lon_str
    
    # First try: decimal format (e.g., ACCESS_CM2_obs_-31.45_117.55_tasmax.csv)
    cache_filename_decimal = f"{model_scenario}_{lat_decimal}_{lon_decimal}_{variable}.csv"
    cache_path_decimal = os.path.join(output_dir, cache_filename_decimal)
    
    # Check if decimal format exists
    if os.path.exists(cache_path_decimal):
        return cache_path_decimal
    
    # Second try: underscore format (e.g., ACCESS_CM2_obs_neg31_45_117_55_tasmax.csv)
    cache_filename_underscore = f"{model_scenario}_{lat_str}_{lon_str}_{variable}.csv"
    cache_path_underscore = os.path.join(output_dir, cache_filename_underscore)
    
    # Return the decimal format path (even if it doesn't exist yet) as it matches existing files
    # The load_cached_variable function will handle the file not found case
    return cache_path_decimal


def load_cached_variable(cache_path):
    """
    Load cached variable data from CSV file.
    
    Parameters:
    -----------
    cache_path : str
        Path to cached CSV file
    
    Returns:
    --------
    pd.DataFrame or None
        DataFrame with date and value columns if file exists and is valid, None otherwise
    """
    if not os.path.exists(cache_path):
        return None
    
    try:
        df = pd.read_csv(cache_path)
        if 'date' not in df.columns or 'value' not in df.columns:
            print(f"  [WARNING] Cached file missing required columns, will re-extract")
            return None
        
        df['date'] = pd.to_datetime(df['date'])
        # Normalize dates to date-only (remove time component) to ensure proper merging
        df['date'] = df['date'].dt.normalize()
        
        # Basic validation - check if file has data
        if len(df) == 0:
            print(f"  [WARNING] Cached file is empty, will re-extract")
            return None
        
        # Check if values are all zero (might indicate a problem)
        if 'value' in df.columns:
            non_zero_count = (df['value'] != 0).sum()
            if non_zero_count == 0 and len(df) > 0:
                print(f"  [WARNING] All values in cache are zero! This may indicate a problem with the cache file.")
            else:
                print(f"  [INFO] Cache loaded: {len(df):,} records, {non_zero_count:,} non-zero values")
        
        return df
    
    except Exception as e:
        print(f"  [WARNING] Error loading cached file: {e}, will re-extract")
        return None


def save_cached_variable(df, cache_path):
    """
    Save extracted variable data to CSV cache file.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with date and value columns
    cache_path : str
        Path to save cached CSV file
    """
    try:
        df[['date', 'value']].to_csv(
            cache_path,
            index=False,
            encoding='utf-8',
            float_format='%.6f'
        )
        print(f"  [INFO] Saved to cache: {os.path.basename(cache_path)}")
    except Exception as e:
        print(f"  [WARNING] Failed to save cache: {e}")



In [53]:
def load_vp_from_csv(output_dir, model_scenario, lat_str, lon_str):
    """
    Load Vapor Pressure (VP) data from CSV file generated by CMPI6_VP_Calculation.ipynb.
    
    This function searches for VP CSV files with flexible naming patterns to handle
    different coordinate formatting conventions.
    
    Parameters:
    -----------
    output_dir : str
        Directory containing the VP CSV file
    model_scenario : str
        Model and scenario string (e.g., "ACCESS_CM2_SSP245")
    lat_str : str
        Latitude formatted as string (e.g., "neg31_75")
    lon_str : str
        Longitude formatted as string (e.g., "117_60")
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with date and value (vapor pressure hPa) columns, or None if file not found
    """
    # Try new format first (decimal coordinates with neg prefix: neg31.75_117.60)
    # Convert from underscore format to decimal format, keeping 'neg' prefix
    # If lat_str uses underscores (neg31_75), convert to neg31.75
    if '_' in lat_str and 'neg' in lat_str:
        lat_str_decimal = lat_str.replace('_', '.').replace('neg', 'neg')
    elif '_' in lat_str:
        lat_str_decimal = lat_str.replace('_', '.').replace('-', 'neg')
    else:
        lat_str_decimal = lat_str.replace('-', 'neg')
    lon_str_decimal = lon_str.replace('_', '.')
    vp_filename_new = f"{model_scenario}_{lat_str_decimal}_{lon_str_decimal}_vp.csv"
    vp_path = os.path.join(output_dir, vp_filename_new)
    
    # If new format doesn't exist, try old format (neg31_75_117_60)
    if not os.path.exists(vp_path):
        vp_filename_old = f"{model_scenario}_{lat_str}_{lon_str}_vp.csv"
        vp_path = os.path.join(output_dir, vp_filename_old)
        
        # If still not found, search for any matching VP file
        if not os.path.exists(vp_path):
            matching_files = glob.glob(os.path.join(output_dir, f"{model_scenario}*_vp.csv"))
            if matching_files:
                vp_path = matching_files[0]
                print(f"  [INFO] Found VP file with alternative naming: {os.path.basename(vp_path)}")
            else:
                print(f"  [WARNING] VP CSV file not found")
                print(f"  [INFO] Expected formats:")
                print(f"    - New format: {vp_filename_new}")
                print(f"    - Old format: {vp_filename_old}")
                print(f"  [INFO] Searched in: {output_dir}")
                print(f"  [INFO] Please run CMPI6_VP_Calculation.ipynb first to generate the VP data")
                return None
    
    try:
        vp_df = pd.read_csv(vp_path)
        vp_df['date'] = pd.to_datetime(vp_df['date'])
        print(f"  [OK] Loaded VP data from: {os.path.basename(vp_path)}")
        print(f"  [INFO] VP records: {len(vp_df):,} days")
        print(f"  [INFO] Date range: {vp_df['date'].min()} to {vp_df['date'].max()}")
        print(f"  [INFO] VP range: {vp_df['value'].min():.2f} to {vp_df['value'].max():.2f} hPa")
        print(f"  [SUCCESS] VP data loaded successfully!")
        return vp_df
    except Exception as e:
        print(f"  [ERROR] Failed to load VP CSV file: {e}")
        return None


def load_evap_from_csv(output_dir, model_scenario, lat_str, lon_str):
    """
    Load Evaporation (Evap) data from CSV file generated by CMPI6_Evap_Calculation.ipynb.
    
    Parameters:
    -----------
    output_dir : str
        Directory containing the Evap CSV file
    model_scenario : str
        Model and scenario string (e.g., "ACCESS_CM2_SSP245")
    lat_str : str
        Latitude formatted as string (e.g., "neg31_75")
    lon_str : str
        Longitude formatted as string (e.g., "117_60")
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with date and value (evaporation mm/day) columns, or None if file not found
    """
    # Try new format first (decimal coordinates with neg prefix: neg31.75_117.60)
    # Convert from underscore format to decimal format, keeping 'neg' prefix
    # If lat_str uses underscores (neg31_75), convert to neg31.75
    if '_' in lat_str and 'neg' in lat_str:
        lat_str_decimal = lat_str.replace('_', '.').replace('neg', 'neg')
    elif '_' in lat_str:
        lat_str_decimal = lat_str.replace('_', '.').replace('-', 'neg')
    else:
        lat_str_decimal = lat_str.replace('-', 'neg')
    lon_str_decimal = lon_str.replace('_', '.')
    
    # Try _eto.csv first (actual filename from CMPI6_Evap_Calculation.ipynb)
    evap_filename_eto = f"{model_scenario}_{lat_str_decimal}_{lon_str_decimal}_eto.csv"
    evap_path = os.path.join(output_dir, evap_filename_eto)
    
    # If _eto.csv doesn't exist, try _evap.csv (alternative naming)
    if not os.path.exists(evap_path):
        evap_filename_evap = f"{model_scenario}_{lat_str_decimal}_{lon_str_decimal}_evap.csv"
        evap_path = os.path.join(output_dir, evap_filename_evap)
    
    # If still not found, try old format with underscores (neg31_75_117_60)
    if not os.path.exists(evap_path):
        evap_filename_old_eto = f"{model_scenario}_{lat_str}_{lon_str}_eto.csv"
        evap_path = os.path.join(output_dir, evap_filename_old_eto)
    
    if not os.path.exists(evap_path):
        evap_filename_old_evap = f"{model_scenario}_{lat_str}_{lon_str}_evap.csv"
        evap_path = os.path.join(output_dir, evap_filename_old_evap)
    
    # If still not found, search for any matching Evap/ET₀ file
    if not os.path.exists(evap_path):
        matching_files = glob.glob(os.path.join(output_dir, f"{model_scenario}*_evap.csv"))
        matching_files.extend(glob.glob(os.path.join(output_dir, f"{model_scenario}*_eto.csv")))
        if matching_files:
            evap_path = matching_files[0]
            print(f"  [INFO] Found Evap/ET₀ file with alternative naming: {os.path.basename(evap_path)}")
        else:
            print(f"  [WARNING] Evap CSV file not found")
            print(f"  [INFO] Expected formats:")
            print(f"    - ET₀ format: {evap_filename_eto}")
            print(f"    - Evap format: {evap_filename_evap}")
            print(f"    - Old ET₀ format: {evap_filename_old_eto}")
            print(f"    - Old Evap format: {evap_filename_old_evap}")
            print(f"  [INFO] Searched in: {output_dir}")
            print(f"  [INFO] Please run CMPI6_Evap_Calculation.ipynb first to generate the ET₀/Evap data")
            return None
    
    try:
        evap_df = pd.read_csv(evap_path)
        evap_df['date'] = pd.to_datetime(evap_df['date'])
        print(f"  [OK] Loaded Evap data from: {os.path.basename(evap_path)}")
        print(f"  [INFO] Evap records: {len(evap_df):,} days")
        print(f"  [INFO] Date range: {evap_df['date'].min()} to {evap_df['date'].max()}")
        print(f"  [INFO] Evap range: {evap_df['value'].min():.2f} to {evap_df['value'].max():.2f} mm/day")
        print(f"  [SUCCESS] Evap data loaded successfully!")
        return evap_df
    except Exception as e:
        print(f"  [ERROR] Failed to load Evap CSV file: {e}")
        return None

## Section 4: MET File Creation Functions

In [54]:
def calculate_tav_amp(df):
    """
    Calculate annual average temperature (tav) and annual amplitude (amp).
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with date index and maxt, mint columns
    
    Returns:
    --------
    tuple: (tav, amp)
    """
    # Calculate mean temperature for each day
    df['tmean'] = (df['maxt'] + df['mint']) / 2.0
    
    # Group by month and calculate monthly means
    df['month'] = df.index.month
    monthly_means = df.groupby('month')['tmean'].mean()
    
    # Calculate annual average (tav)
    tav = monthly_means.mean()
    
    # Calculate annual amplitude (amp)
    # amp = (max monthly mean - min monthly mean) / 2
    amp = (monthly_means.max() - monthly_means.min()) / 2.0
    
    return tav, amp

In [55]:
def create_complete_met_file(tasmax_df, tasmin_df, pr_df, rsds_df, vp_df, evap_df, 
                            scenario=None, output_dir=None, latitude=None, 
                            longitude=None, model=None, start_year=None, end_year=None):
    """
    Create ACCESS MET format file with all variables including VP and Evap.
    
    Parameters:
    -----------
    tasmax_df : pd.DataFrame
        DataFrame with date and value (maximum temperature °C) columns
    tasmin_df : pd.DataFrame
        DataFrame with date and value (minimum temperature °C) columns
    pr_df : pd.DataFrame
        DataFrame with date and value (precipitation mm) columns
    rsds_df : pd.DataFrame
        DataFrame with date and value (solar radiation W/m²) columns
    vp_df : pd.DataFrame
        DataFrame with date and value (vapor pressure hPa) columns
    evap_df : pd.DataFrame
        DataFrame with date and value (evaporation mm/day) columns
    scenario : str
        Scenario name (e.g., "SSP245", "SSP585")
    output_dir : str
        Output directory path
    latitude : float
        Latitude in decimal degrees
    longitude : float
        Longitude in decimal degrees
    model : str
        Model name (e.g., "ACCESS CM2")
    start_year : int, optional
        Start year for MET file. Only data from this year onwards will be included.
        If None, uses the minimum date from the data.
    end_year : int, optional
        End year for MET file. Only data up to this year will be included.
        If None, uses the maximum date from the data.
    
    Returns:
    --------
    tuple: (tav, amp, num_rows)
    """
    # Merge all dataframes on date
    # Normalize all dates to date-only (remove time component) to ensure proper merging
    merged = tasmax_df.copy()
    merged = merged.rename(columns={'value': 'maxt'})
    merged['date'] = pd.to_datetime(merged['date']).dt.normalize()
    print(f"  [DEBUG] tasmax: {len(merged):,} records, maxt range: {merged['maxt'].min():.2f} to {merged['maxt'].max():.2f}")
    
    # Merge tasmin
    tasmin_df['date'] = pd.to_datetime(tasmin_df['date']).dt.normalize()
    merged = merged.merge(tasmin_df[['date', 'value']], on='date', how='outer')
    merged = merged.rename(columns={'value': 'mint'})
    print(f"  [DEBUG] After tasmin merge: {len(merged):,} records, mint range: {merged['mint'].min():.2f} to {merged['mint'].max():.2f}")
    
    # Merge pr (precipitation/rain)
    pr_df['date'] = pd.to_datetime(pr_df['date']).dt.normalize()
    merged = merged.merge(pr_df[['date', 'value']], on='date', how='outer')
    merged = merged.rename(columns={'value': 'rain'})
    print(f"  [DEBUG] After pr merge: {len(merged):,} records, rain range: {merged['rain'].min():.2f} to {merged['rain'].max():.2f}")
    
    # Merge rsds (radiation) - REQUIRED
    rsds_df['date'] = pd.to_datetime(rsds_df['date']).dt.normalize()
    print(f"  [DEBUG] rsds before merge: {len(rsds_df):,} records, value range: {rsds_df['value'].min():.2f} to {rsds_df['value'].max():.2f}")
    
    # Check if rsds data has any non-zero values (diagnostic)
    rsds_nonzero = (rsds_df['value'] != 0).sum()
    rsds_total = len(rsds_df)
    if rsds_nonzero == 0 and rsds_total > 0:
        print(f"  [WARNING] All rsds values are zero in source data! This may indicate missing/corrupted rsds NetCDF files.")
    elif rsds_nonzero < rsds_total * 0.1:
        print(f"  [WARNING] Only {rsds_nonzero:,} out of {rsds_total:,} rsds values are non-zero ({100*rsds_nonzero/rsds_total:.1f}%)")
    
    # Convert W/m2 to MJ/m2 by multiplying by 0.0864 (seconds per day / 1e6)
    rsds_df['value_mj'] = rsds_df['value'] * 0.0864
    merged = merged.merge(rsds_df[['date', 'value_mj']], on='date', how='outer')
    merged = merged.rename(columns={'value_mj': 'radn'})
    print(f"  [DEBUG] After rsds merge: {len(merged):,} records, radn range: {merged['radn'].min():.2f} to {merged['radn'].max():.2f}")
    print(f"  [DEBUG] Non-null counts - maxt: {merged['maxt'].notna().sum()}, mint: {merged['mint'].notna().sum()}, rain: {merged['rain'].notna().sum()}, radn: {merged['radn'].notna().sum()}")
    
    # Diagnostic: Check radn values after merge
    radn_nonzero_after_merge = (merged['radn'] != 0).sum() if 'radn' in merged.columns else 0
    radn_total_after_merge = merged['radn'].notna().sum() if 'radn' in merged.columns else 0
    if radn_total_after_merge > 0:
        print(f"  [INFO] After merge: {radn_nonzero_after_merge:,} non-zero radn values out of {radn_total_after_merge:,} total ({100*radn_nonzero_after_merge/radn_total_after_merge:.1f}%)")
    
    # Merge VP (vapor pressure)
    vp_df['date'] = pd.to_datetime(vp_df['date']).dt.normalize()
    merged = merged.merge(vp_df[['date', 'value']], on='date', how='left')
    merged = merged.rename(columns={'value': 'vp'})
    merged['vp'] = pd.to_numeric(merged['vp'], errors='coerce')
    
    # Merge Evap (evaporation)
    evap_df['date'] = pd.to_datetime(evap_df['date']).dt.normalize()
    merged = merged.merge(evap_df[['date', 'value']], on='date', how='left')
    merged = merged.rename(columns={'value': 'evap'})
    merged['evap'] = pd.to_numeric(merged['evap'], errors='coerce')
    
    # Sort by date
    merged = merged.sort_values('date').reset_index(drop=True)
    
    # Diagnostic: Check radn values before filtering
    if 'radn' in merged.columns and len(merged) > 0:
        radn_nonzero_before = (merged['radn'] != 0).sum()
        radn_total_before = merged['radn'].notna().sum()
        radn_min = merged['radn'].min() if radn_total_before > 0 else None
        radn_max = merged['radn'].max() if radn_total_before > 0 else None
        if radn_total_before > 0:
            print(f"  [INFO] Before filtering: {radn_nonzero_before:,} non-zero radn values out of {radn_total_before:,} total")
            if radn_min is not None and radn_max is not None:
                print(f"  [INFO] Radn range before filtering: {radn_min:.2f} to {radn_max:.2f} MJ/m²")
        else:
            print(f"  [WARNING] No radn data available before filtering!")
    
    # Check actual data date range before filtering and create backup
    merged_backup = merged.copy()  # Backup in case filtering removes all data
    original_count_before_filter = len(merged)
    data_min_date = merged['date'].min() if len(merged) > 0 else None
    data_max_date = merged['date'].max() if len(merged) > 0 else None
    original_date_range = f"{data_min_date} to {data_max_date}" if data_min_date is not None else "no data"
    
    # Check if specified date range overlaps with actual data
    use_actual_date_range = False
    if start_year is not None and data_max_date is not None:
        specified_start = pd.Timestamp(year=start_year, month=1, day=1)
        if specified_start > data_max_date:
            print(f"  [WARNING] Specified START_YEAR={start_year} is after data end date ({data_max_date.year})")
            print(f"  [WARNING] Data date range: {original_date_range}")
            print(f"  [INFO] Using actual data date range instead of specified START_YEAR/END_YEAR")
            use_actual_date_range = True
    if end_year is not None and data_min_date is not None:
        specified_end = pd.Timestamp(year=end_year, month=12, day=31)
        if specified_end < data_min_date:
            print(f"  [WARNING] Specified END_YEAR={end_year} is before data start date ({data_min_date.year})")
            print(f"  [WARNING] Data date range: {original_date_range}")
            print(f"  [INFO] Using actual data date range instead of specified START_YEAR/END_YEAR")
            use_actual_date_range = True
    
    # Filter data to only include dates from start_year onwards and up to end_year
    # But skip filtering if date range doesn't overlap with data
    if not use_actual_date_range:
        if start_year is not None:
            start_date = pd.Timestamp(year=start_year, month=1, day=1)
            merged = merged[merged['date'] >= start_date].copy()
            removed_count = original_count_before_filter - len(merged)
            if removed_count > 0:
                print(f"  [INFO] Filtered data to start from {start_year}: removed {removed_count:,} earlier records")
                if len(merged) == 0:
                    print(f"  [WARNING] All data was filtered out by START_YEAR={start_year}!")
                    print(f"  [WARNING] Restoring data and using actual data date range instead")
                    merged = merged_backup.copy()  # Restore from backup
                    use_actual_date_range = True
        
        if end_year is not None and len(merged) > 0:
            end_date = pd.Timestamp(year=end_year, month=12, day=31)
            count_before_end_filter = len(merged)
            merged = merged[merged['date'] <= end_date].copy()
            removed_count = count_before_end_filter - len(merged)
            if removed_count > 0:
                print(f"  [INFO] Filtered data to end at {end_year}: removed {removed_count:,} later records")
                if len(merged) == 0:
                    print(f"  [WARNING] All data was filtered out by END_YEAR={end_year}!")
                    print(f"  [WARNING] Restoring data and using actual data date range instead")
                    merged = merged_backup.copy()  # Restore from backup
                    use_actual_date_range = True
    
    # Diagnostic: Check radn values after filtering
    if 'radn' in merged.columns and len(merged) > 0:
        radn_nonzero_after_filter = (merged['radn'] != 0).sum()
        radn_total_after_filter = merged['radn'].notna().sum()
        if radn_total_after_filter > 0:
            print(f"  [INFO] After filtering: {radn_nonzero_after_filter:,} non-zero radn values out of {radn_total_after_filter:,} total")
        else:
            print(f"  [WARNING] No radn data remaining after filtering!")
    
    # CRITICAL: Create a complete date range to ensure all days are included
    # Use actual data range if filtering removed all data or date range doesn't overlap
    if use_actual_date_range:
        # Update dates from restored/current merged data
        if len(merged) > 0:
            data_min_date = merged['date'].min()
            data_max_date = merged['date'].max()
        if data_min_date is not None and data_max_date is not None:
            # Use actual data date range
            min_date = pd.Timestamp(year=data_min_date.year, month=1, day=1)
            max_date = pd.Timestamp(year=data_max_date.year, month=12, day=31)
            print(f"  [INFO] Using actual data date range: {min_date.year} to {max_date.year}")
        else:
            # Fallback
            min_date = pd.Timestamp(year=1985, month=1, day=1)
            max_date = pd.Timestamp(year=2014, month=12, day=31)
    elif start_year is not None and len(merged) > 0:
        min_date = pd.Timestamp(year=start_year, month=1, day=1)
        if end_year is not None:
            max_date = pd.Timestamp(year=end_year, month=12, day=31)
        else:
            max_date = merged['date'].max() if len(merged) > 0 else pd.Timestamp(year=2014, month=12, day=31)
            max_date = pd.Timestamp(year=max_date.year, month=12, day=31)
    else:
        # Use data range
        if len(merged) > 0:
            min_date = merged['date'].min()
            min_date = pd.Timestamp(year=min_date.year, month=1, day=1)
            max_date = merged['date'].max()
            max_date = pd.Timestamp(year=max_date.year, month=12, day=31)
        else:
            # Fallback if no data
            min_date = pd.Timestamp(year=1985, month=1, day=1)
            max_date = pd.Timestamp(year=2014, month=12, day=31)
    
    original_count = len(merged)
    
    # Create complete date range (includes all days, including day 366 for leap years)
    complete_date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    # Ensure the date range index has a name so reset_index() works correctly
    complete_date_range.name = 'date'
    
    # Set date as index for reindexing
    merged = merged.set_index('date')
    
    # Reindex to include all days in the complete range
    merged = merged.reindex(complete_date_range)
    # Ensure index name is preserved after reindex (critical for reset_index to work)
    merged.index.name = 'date'
    # Verify the name was set correctly
    if merged.index.name != 'date':
        # Force set it again
        merged.index = merged.index.rename('date')
    
    # Report if days were added
    new_count = len(merged)
    if new_count > original_count:
        print(f"  [INFO] Added {new_count - original_count} missing days to ensure complete date range")
        # Check if radn is all NaN after reindexing (indicates no rsds data for the date range)
        if 'radn' in merged.columns:
            radn_nan_count = merged['radn'].isna().sum()
            if radn_nan_count == new_count - original_count:
                print(f"  [WARNING] All {radn_nan_count} added days have NaN radn values - no rsds data available for this date range!")
                print(f"  [WARNING] This will result in radn=0.0 after filling. Check if rsds data exists for the specified date range.")
    
    # Fill missing values for numeric columns using forward fill then backward fill
    # But detect and warn about constant values (indicates missing/corrupted data)
    numeric_cols = ['maxt', 'mint', 'rain', 'radn']
    for col in numeric_cols:
        if col in merged.columns:
            # Check for constant values AFTER reindex but BEFORE filling
            # This helps identify years that were completely missing and got filled with constant values
            merged['year_temp'] = merged.index.year
            for year in merged['year_temp'].unique():
                year_mask = merged['year_temp'] == year
                year_data = merged.loc[year_mask, col]
                # Check if all non-NaN values are the same (constant value)
                non_null_data = year_data.dropna()
                if len(non_null_data) > 10:  # Only warn if we have enough data points
                    if non_null_data.nunique() == 1:
                        constant_value = non_null_data.iloc[0]
                        # Count how many days have this value
                        num_constant_days = (year_data == constant_value).sum()
                        if num_constant_days >= 300:  # Most of the year is constant
                            print(f"  [WARNING] Year {year} has constant {col} value ({constant_value:.2f}) for {num_constant_days} days - likely missing/corrupted source data")
            merged = merged.drop(columns=['year_temp'])
            
            # Fill missing values (but this won't help if the source data itself is constant)
            merged[col] = merged[col].ffill().bfill()
            merged[col] = merged[col].fillna(0.0)
    
    # Handle VP separately - copy from day 365 to day 366 for leap years
    if 'vp' in merged.columns:
        # Reset index to get date back as a column
        original_index_name = merged.index.name  # Capture BEFORE reset_index
        merged_temp = merged.reset_index()
        
        # Ensure 'date' column exists - handle all possible cases
        if 'date' not in merged_temp.columns:
            # Check if 'index' column exists (created when index had no name)
            if 'index' in merged_temp.columns:
                merged_temp = merged_temp.rename(columns={'index': 'date'})
            # Check if original index name exists as a column
            elif original_index_name and original_index_name in merged_temp.columns:
                merged_temp = merged_temp.rename(columns={original_index_name: 'date'})
            else:
                # Find the first datetime column (should be the index that was reset)
                for col in merged_temp.columns:
                    if pd.api.types.is_datetime64_any_dtype(merged_temp[col]):
                        merged_temp = merged_temp.rename(columns={col: 'date'})
                        break
        
        # Final safety check - if still no 'date' column, find the column created by reset_index
        if 'date' not in merged_temp.columns:
            # Find any datetime column (should be the one created by reset_index)
            datetime_cols = [col for col in merged_temp.columns if pd.api.types.is_datetime64_any_dtype(merged_temp[col])]
            if datetime_cols:
                # Use the first datetime column (should be the index that was reset)
                merged_temp = merged_temp.rename(columns={datetime_cols[0]: 'date'})
            else:
                # Absolute last resort: create from the original merged index
                if len(merged.index) == len(merged_temp):
                    merged_temp['date'] = pd.to_datetime(merged.index.values)
                else:
                    raise ValueError(f"Cannot create 'date' column. Original index name: {original_index_name}, "
                                   f"merged_temp columns: {list(merged_temp.columns)}, "
                                   f"merged index type: {type(merged.index)}, "
                                   f"Lengths - merged: {len(merged)}, merged_temp: {len(merged_temp)}")
        
        # Verify 'date' column exists before using it
        if 'date' not in merged_temp.columns:
            raise ValueError(f"Could not create 'date' column in merged_temp. Available columns: {list(merged_temp.columns)}")
        
        merged_temp['year'] = merged_temp['date'].dt.year
        merged_temp['day'] = merged_temp['date'].dt.dayofyear
        
        # Find all day 366 rows (leap year days)
        day366_mask = merged_temp['day'] == 366
        day366_rows = merged_temp[day366_mask].copy()
        
        if len(day366_rows) > 0:
            # For each day 366, copy VP from day 365 of the same year
            for idx in day366_rows.index:
                year = merged_temp.loc[idx, 'year']
                # Find day 365 of the same year
                day365_mask = (merged_temp['year'] == year) & (merged_temp['day'] == 365)
                day365_rows = merged_temp[day365_mask]
                
                if len(day365_rows) > 0:
                    # Copy VP value from day 365 to day 366
                    vp_day365 = merged_temp.loc[day365_rows.index[0], 'vp']
                    vp_day366 = merged_temp.loc[idx, 'vp']
                    if pd.isna(vp_day366) or vp_day366 == '':
                        merged_temp.loc[idx, 'vp'] = vp_day365
                        print(f"  [INFO] Copied VP value from day 365 to day 366 for year {year}: {vp_day365:.2f} hPa")
        
        # Set date back as index
        merged = merged_temp.set_index('date')
        merged = merged.drop(columns=['year', 'day'])
    
    # Fill missing VP and Evap values
    # IMPORTANT: Only forward fill to avoid propagating values backward to earlier dates
    # This prevents constant values in the first years when VP/Evap data starts later than base variables
    # If VP/Evap data is missing at the start, leave it as NaN (don't backward fill)
    if 'vp' in merged.columns:
        # Forward fill only - fills gaps within the data range but doesn't fill backward
        merged['vp'] = merged['vp'].ffill()
    if 'evap' in merged.columns:
        # Forward fill only - fills gaps within the data range but doesn't fill backward
        merged['evap'] = merged['evap'].ffill()
    
    # Reset index to get date back as a column
    original_index_name = merged.index.name  # Capture BEFORE reset_index
    merged = merged.reset_index()
    
    # Ensure 'date' column exists - handle all possible cases
    if 'date' not in merged.columns:
        # Check if 'index' column exists (created when index had no name)
        if 'index' in merged.columns:
            merged = merged.rename(columns={'index': 'date'})
        # Check if original index name exists as a column
        elif original_index_name and original_index_name in merged.columns:
            merged = merged.rename(columns={original_index_name: 'date'})
        else:
            # Find the first datetime column (should be the index that was reset)
            for col in merged.columns:
                if pd.api.types.is_datetime64_any_dtype(merged[col]):
                    merged = merged.rename(columns={col: 'date'})
                    break
    
    # Final safety check - if still no 'date' column, find datetime column or create from index
    if 'date' not in merged.columns:
        # Find any datetime column
        datetime_cols = [col for col in merged.columns if pd.api.types.is_datetime64_any_dtype(merged[col])]
        if datetime_cols:
            merged = merged.rename(columns={datetime_cols[0]: 'date'})
        else:
            # Last resort: use the index itself (should be datetime after reset_index)
            if isinstance(merged.index, pd.DatetimeIndex):
                merged['date'] = merged.index
            else:
                # Convert index to datetime if possible
                merged['date'] = pd.to_datetime(merged.index, errors='coerce')
    
    # Calculate tav and amp (use only non-NaN values for calculation)
    merged_temp = merged[['date', 'maxt', 'mint']].copy()
    merged_temp = merged_temp.set_index('date')
    merged_temp.index = pd.to_datetime(merged_temp.index)
    tav, amp = calculate_tav_amp(merged_temp)
    
    # Verify 'date' column exists before using it
    if 'date' not in merged.columns:
        raise ValueError(f"Could not create 'date' column in merged. Available columns: {list(merged.columns)}")
    
    # Create year and day columns
    merged['year'] = merged['date'].dt.year
    merged['day'] = merged['date'].dt.dayofyear
    
    # Add code column (hardcoded to '222222')
    merged['code'] = '222222'
    
    # Create met_data
    met_data = merged[['year', 'day', 'radn', 'maxt', 'mint', 'rain', 'evap', 'vp', 'code']].copy()
    
    # #region agent log
    with open(r"c:\Users\ibian\Desktop\ClimAdapt\Anameka\Grid\Anameka\.cursor\debug.log", "a") as f:
        import json
        log_entry = {"location":"create_complete_met_file:904","message":"met_data created","data":{"columns":list(met_data.columns),"index_name":str(met_data.index.name),"has_date_col":"date" in met_data.columns},"timestamp":int(time.time()*1000),"sessionId":"debug-session","runId":"run1","hypothesisId":"A"}
        f.write(json.dumps(log_entry) + "\n")
    # #endregion agent log
    
    # Ensure numeric columns are properly formatted
    for col in ['radn', 'maxt', 'mint', 'rain', 'evap', 'vp']:
        if col in met_data.columns:
            met_data[col] = pd.to_numeric(met_data[col], errors='coerce')
    
    # Prepare header
    current_date = datetime.now().strftime('%Y%m%d')
    model_scenario = f"{model.replace(' ', '_')}_{scenario}" if model and scenario else "CMIP6"
    
    header = f"""[weather.met.weather]
!Your Ref:  "
latitude = {latitude:.2f}  (DECIMAL DEGREES)
longitude =  {longitude:.2f}  (DECIMAL DEGREES)
tav = {tav:.2f} (oC) ! Annual average ambient temperature.
amp = {amp:.2f} (oC) ! Annual amplitude in mean monthly temperature.
!Data Extracted from CMIP6 {model} {scenario} dataset on {current_date} for APSIM
!As evaporation is read at 9am, it has been shifted to day before
!ie The evaporation measured on 20 April is in row for 19 April
!The 6 digit code indicates the source of the 6 data columns
!0 actual observation, 1 actual observation composite station
!2 interpolated from daily observations
!3 interpolated from daily observations using anomaly interpolation method for CLIMARC data
!6 synthetic pan
!7 interpolated long term averages
!more detailed two digit codes are available in SILO's 'Standard' format files
!
!For further information see the documentation on the datadrill
!  http://www.longpaddock.qld.gov.au/silo
!
year  day radn  maxt   mint  rain  evap    vp   code
 ()   () (MJ/m^2) (oC)  (oC)  (mm)  (mm) (hPa)     ()
"""
    
    # Create output filename with coordinate-based naming
    # Format lat_str to use 'neg' prefix instead of '-' for filenames
    lat_str = f"{latitude:.2f}".replace('-', 'neg')
    lon_str = f"{longitude:.2f}"
    output_filename = f"{model_scenario}_{lat_str}_{lon_str}.met"
    output_path = os.path.join(output_dir, output_filename)
    
    # Write MET file
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(header)
        # Write data rows
        for _, row in met_data.iterrows():
            # Format the row with proper spacing
            radn_val = row['radn'] if pd.notna(row['radn']) else 0.0
            evap_val = row['evap'] if pd.notna(row['evap']) else 0.0
            vp_val = row['vp'] if pd.notna(row['vp']) else 0.0
            
            # Format numbers with proper spacing
            radn_str = f"{float(radn_val):6.1f}"
            evap_str = f"{float(evap_val):6.1f}"
            vp_str = f"{float(vp_val):6.1f}"
            
            # Code is hardcoded to '222222'
            code_str = "222222"
            
            # Handle NaN values for maxt, mint, rain - use 0.0 as default
            maxt_val = row['maxt'] if pd.notna(row['maxt']) else 0.0
            mint_val = row['mint'] if pd.notna(row['mint']) else 0.0
            rain_val = row['rain'] if pd.notna(row['rain']) else 0.0
            
            # Format with proper column widths
            line = f"{int(row['year']):4d} {int(row['day']):4d} {radn_str} {maxt_val:6.1f} {mint_val:6.1f} {rain_val:6.1f} {evap_str} {vp_str} {code_str}\n"
            f.write(line)
    
    # Also create CSV version (using same lat_str format with 'neg')
    csv_filename = f"{model_scenario}_{lat_str}_{lon_str}.csv"
    csv_path = os.path.join(output_dir, csv_filename)
    met_data.to_csv(csv_path, index=False, encoding='utf-8', float_format='%.1f', na_rep='')
    
    num_rows = len(met_data)
    
    print(f"  [OK] Created MET file: {output_filename}")
    print(f"  [OK] Created CSV file: {csv_filename}")
    print(f"  [OK] Total rows: {num_rows:,}")
    
    # #region agent log
    with open(r"c:\Users\ibian\Desktop\ClimAdapt\Anameka\Grid\Anameka\.cursor\debug.log", "a") as f:
        import json
        log_entry = {"location":"create_complete_met_file:983","message":"Before date range access","data":{"columns":list(met_data.columns),"index_name":str(met_data.index.name),"has_date_col":"date" in met_data.columns,"index_type":str(type(met_data.index))},"timestamp":int(time.time()*1000),"sessionId":"debug-session","runId":"run1","hypothesisId":"A"}
        f.write(json.dumps(log_entry) + "\n")
    # #endregion agent log
    
    # Use index for date range since 'date' is the index, not a column
    # met_data inherits the index from merged, which should be a DatetimeIndex
    try:
        if isinstance(met_data.index, pd.DatetimeIndex):
            print(f"  [OK] Date range: {met_data.index.min()} to {met_data.index.max()}")
        elif 'date' in met_data.columns:
            print(f"  [OK] Date range: {met_data['date'].min()} to {met_data['date'].max()}")
        else:
            # Try to get date range from merged if available
            print(f"  [INFO] Date range: {num_rows:,} rows (date info not directly available)")
    except Exception as e:
        print(f"  [INFO] Could not determine date range: {e}")
    
    return tav, amp, num_rows

## Section 5: Main Processing - Extract Data and Create ACCESS MET File

In [56]:
# Construct data directory path
data_dir = os.path.join(CMIP6_BASE_DIR, f"{MODEL} {SCENARIO}")

if not os.path.exists(data_dir):
    raise ValueError(f"Data directory not found: {data_dir}")

print("="*70)
print(f"Processing Coordinate: ({LATITUDE:.6f}, {LONGITUDE:.6f})")
print(f"Model: {MODEL}, Scenario: {SCENARIO}")
print("="*70)
print(f"\nData directory: {data_dir}\n")

# Extract data for all variables
extracted_data = {}

# Extract base variables (required for MET file)
all_variables = BASE_VARIABLES + VP_VARIABLES + EVAP_VARIABLES

for variable in all_variables:
    print(f"\n{'='*70}")
    print(f"Processing variable: {variable}")
    print(f"{'='*70}")
    
    # Check for cached data first
    cache_path = get_cached_variable_path(OUTPUT_DIR, model_scenario, lat_str, lon_str, variable)
    print(f"  [DEBUG] Checking cache: {os.path.basename(cache_path)}")
    print(f"  [DEBUG] Full cache path: {cache_path}")
    df = load_cached_variable(cache_path)
    
    if df is not None:
        # Use cached data
        extracted_data[variable] = df
        print(f"  [OK] Loaded from cache: {len(df):,} records for {variable}")
        print(f"  [INFO] Date range: {df['date'].min()} to {df['date'].max()}")
    else:
        # Extract from NetCDF files
        print(f"  [INFO] Cache not found, extracting from NetCDF files...")
        df = extract_daily_data_from_netcdf(
            data_dir, 
            variable, 
            LATITUDE, 
            LONGITUDE, 
            tolerance=COORD_TOLERANCE
        )
        
        if df is not None and len(df) > 0:
            extracted_data[variable] = df
            print(f"  [OK] Extracted {len(df):,} records for {variable}")
            # Save to cache for future use
            save_cached_variable(df, cache_path)
        else:
            if variable in BASE_VARIABLES:
                raise ValueError(f"Failed to extract required variable: {variable}")
            else:
                print(f"  [WARNING] Failed to extract optional variable: {variable}")

# Check if all required base variables are available
missing_vars = [v for v in BASE_VARIABLES if v not in extracted_data]
if missing_vars:
    raise ValueError(f"Missing required variables: {missing_vars}")

print(f"\n{'='*70}")
print("Loading Vapor Pressure (VP) from CSV...")
print(f"{'='*70}")

# Load VP from CSV file generated by CMPI6_VP_Calculation.ipynb
vp_df = load_vp_from_csv(OUTPUT_DIR, model_scenario, lat_str, lon_str)

if vp_df is None:
    print(f"  [WARNING] VP data not available - VP will be set to 0")
    # Create empty VP dataframe with same dates as tasmax
    vp_df = extracted_data['tasmax'].copy()
    vp_df['value'] = 0.0

print(f"\n{'='*70}")
print("Loading Evaporation (Evap) from CSV...")
print(f"{'='*70}")

# Load Evap from CSV file generated by CMPI6_Evap_Calculation.ipynb
evap_df = load_evap_from_csv(OUTPUT_DIR, model_scenario, lat_str, lon_str)

if evap_df is None:
    print(f"  [WARNING] Evap data not available - Evap will be set to 0")
    # Create empty evap dataframe with same dates as tasmax (set to 0)
    evap_df = extracted_data['tasmax'].copy()
    evap_df['value'] = 0.0
    print(f"  [INFO] Created empty evap dataframe with {len(evap_df):,} days (all values = 0.0)")
    print(f"  [INFO] Date range: {evap_df['date'].min()} to {evap_df['date'].max()}\n")

# ============================================================================
# TEMPORARILY COMMENTED OUT: Evap Calculation (uncomment when variables are available)
# ============================================================================
# The evap calculation is temporarily skipped - evap is loaded from CSV instead.
# The evap CSV is generated by CMPI6_Evap_Calculation.ipynb which already incorporates
# all required variables including sfcWind.
# Note: hursmax and hursmin are not available - using hurs (mean relative humidity) instead.

# print(f"\n{'='*70}")
# print("Calculating Evaporation (Evap)...")
# print(f"{'='*70}")
# 
# # Calculate Evap if all required variables are available
# # Note: sfcWind would need to be extracted separately if doing evap calculation here
# evap_required = ['hurs']  # sfcWind already used in CMPI6_Evap_Calculation.ipynb
# evap_available = all(v in extracted_data for v in evap_required)
# 
# if evap_available:
#     # This would require sfcWind to be extracted, but evap is loaded from CSV instead
#     print(f"  [INFO] Evap calculation skipped - loading from CSV instead")
# else:
#     print(f"  [WARNING] Evap variables not available - evap will be set to 0")
#     # Create empty evap dataframe with same dates as tasmax
#     evap_df = extracted_data['tasmax'].copy()
#     evap_df['value'] = 0.0

print(f"\n{'='*70}")
print("Creating ACCESS MET File...")
print(f"{'='*70}")

# Create ACCESS MET file
tav, amp, num_rows = create_complete_met_file(
    tasmax_df=extracted_data['tasmax'],
    tasmin_df=extracted_data['tasmin'],
    pr_df=extracted_data['pr'],
    rsds_df=extracted_data['rsds'],
    vp_df=vp_df,
    evap_df=evap_df,
    scenario=SCENARIO,
    output_dir=OUTPUT_DIR,
    latitude=LATITUDE,
    longitude=LONGITUDE,
    model=MODEL,
    start_year=START_YEAR,
    end_year=END_YEAR
)

print(f"\n{'='*70}")
print("[SUCCESS] ACCESS MET FILE CREATED!")
print(f"{'='*70}")
print(f"  Output Directory: {OUTPUT_DIR}")
print(f"  Total Rows: {num_rows:,}")
print(f"  TAV: {tav:.2f}°C")
print(f"  AMP: {amp:.2f}°C")

Processing Coordinate: (-31.750000, 117.600000)
Model: ACCESS CM2, Scenario: SSP585

Data directory: C:\Users\ibian\Desktop\ClimAdapt\CMIP6\ACCESS CM2 SSP585


Processing variable: tasmax
  [DEBUG] Checking cache: ACCESS_CM2_SSP585_-31.75_117.60_tasmax.csv
  [DEBUG] Full cache path: C:\Users\ibian\Desktop\ClimAdapt\Anameka\Anameka Climate Files_2901\-31.75_117.60_Climate Files\ACCESS_CM2_SSP585_-31.75_117.60_tasmax.csv
  [INFO] Cache loaded: 10,957 records, 10,957 non-zero values
  [OK] Loaded from cache: 10,957 records for tasmax
  [INFO] Date range: 2035-01-01 00:00:00 to 2064-12-30 00:00:00

Processing variable: tasmin
  [DEBUG] Checking cache: ACCESS_CM2_SSP585_-31.75_117.60_tasmin.csv
  [DEBUG] Full cache path: C:\Users\ibian\Desktop\ClimAdapt\Anameka\Anameka Climate Files_2901\-31.75_117.60_Climate Files\ACCESS_CM2_SSP585_-31.75_117.60_tasmin.csv
  [INFO] Cache loaded: 10,957 records, 10,957 non-zero values
  [OK] Loaded from cache: 10,957 records for tasmin
  [INFO] Date range: 